# Titanic Disaster

![Titanic realna fotka](https://raw.githubusercontent.com/Masterx-AI/Project_Titanic_Survival_Prediction_/main/titanic.jpg)

originalny dataset z Kaggle.com je dostupny tu: https://www.kaggle.com/competitions/titanic

## Data Reorganization

1. chceme spojit dataset `test.csv` s datasetom `train.csv`
2. pozriet sa na duplikaty
3. rozdelit stlpec `Name` na `firstname` a `lastname`

### 1. Spojenie datasetov

In [1]:
from pathlib import Path
import pandas as pd

path = Path('data/titanic')

In [10]:
df_train = pd.read_csv(path / 'train.csv', index_col='PassengerId')
df_test = pd.read_csv(path / 'test.csv', index_col='PassengerId')

zobrazime si rozmery jedneho aj druheho dataset-u pomocou vlastnosti `.shape`:

In [12]:
df_train.shape, df_test.shape

((891, 11), (418, 10))

ked sa pozrieme na ne blizsie, tak vieme, ze:

* dataset `train` ma `891` riadkov a `11` stlpcov
* dataset `test` ma `418` riadkov a `10` stlpcov

po spojeni by sme mali v sucte `891 + 418 = 1309` riadkov. ale mame problem so stlpcami - su rozdielne. nemozeme spojit dva datasety, ktore maju rozdielny pocet stlpcov (iba ak by sme niektore vynechali alebo by se vytvorili prazdny stlpec).

pozrime sa teda na to, aky stlpec nam chyba v dataset `test`:

In [14]:
df_train.columns, df_test.columns

(Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
        'Fare', 'Cabin', 'Embarked'],
       dtype='object'),
 Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
        'Cabin', 'Embarked'],
       dtype='object'))

a je to jasne. chyba nam hned prvy stlpec `Survived`. hodnoty tohto stlpca sa totiz nachadzaju v datasete `gender_submission.csv`. takze ho najprv nacitame:

In [19]:
df_survival = pd.read_csv(path / 'gender_submission.csv', index_col='PassengerId')

spojenie teda bude vyzerat takto:

* najprv spojime datasety `gender_submission.csv` a `test.csv` na zaklade stlpca `PassengerId`
* potom spojime datasety `train.csv` s vyslednym datasetom, ktory vznikol spojenim `gender_submission.csv` a `test.csv` rovnako na zaklade stlpca `PassengerId`

In [21]:
# spojenie datasetov gender_submission.csv a test.csv
df_merged = df_test.merge(df_survival, on='PassengerId', how='left')

In [25]:
# spojenie dataset train.csv a vysledneho datasetu, ktory vznikol v predchadzajucom kroku
# df_merged.merge(df_train, how='outer')
passengers = pd.concat( [df_train, df_merged] )

In [26]:
passengers

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [57]:
passengers['lastname'] = passengers['Name'].str.split(',', expand=True)[0]
passengers['firstname'] = passengers['Name'].str.split(',', expand=True)[1]

In [58]:
passengers

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,lastname,firstname
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Mr. Owen Harris
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Miss. Laina
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mrs. Jacques Heath (Lily May Peel)
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Allen,Mr. William Henry
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Spector,Mr. Woolf
1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Oliva y Ocana,Dona. Fermina
1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Saether,Mr. Simon Sivertsen


## Otazky ohladom Titanku

1. Koľko detí bolo na Titaniku? (do 18 rokov)
2. Koľko žien prežilo a koľko mužov prežilo? A koľko mužov a koľko žien tam vlastne bolo?
3. Najstarší a najmladší pasažier? Najmladšia a najstaršia žena/muž?
4. Koľko pasažierov prežilo z jednotlivých tried?
5. Koľko bolo na palube Jackov? A koľko tam bolo Rose?
6. Koľko bola priemerná cena lístka pre príslušnú triedu pasažierov?
7. Koľko pasažierov nastúpilo v jednotlivých prístavoch?
8. Čo všetko malo vplyv na to, že pasažier prežil? (vek, pohlavie, vrstva, na ktorej bol palube, ...)

In [60]:
passengers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 1 to 1309
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Survived   1309 non-null   int64  
 1   Pclass     1309 non-null   int64  
 2   Name       1309 non-null   object 
 3   Sex        1309 non-null   object 
 4   Age        1046 non-null   float64
 5   SibSp      1309 non-null   int64  
 6   Parch      1309 non-null   int64  
 7   Ticket     1309 non-null   object 
 8   Fare       1308 non-null   float64
 9   Cabin      295 non-null    object 
 10  Embarked   1307 non-null   object 
 11  lastname   1309 non-null   object 
 12  firstname  1309 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 175.5+ KB
